In [13]:
import json
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

# === Load and Flatten JSON ===
with open("physics_test.json", "r") as f:
    data = json.load(f)

records = []
for subject in data['subjects']:
    for chapter in subject['chapters']:
        for subtopic in chapter['subtopics']:
            for q_type, questions in subtopic['questions'].items():
                for q in questions:
                    question_text = q.get('questionText') or q.get('question') or q.get('term')
                    correct_answer = q.get('correctAnswer') or q.get('correctDefinition')
                    options = q.get('options') or q.get('definitionOptions') or []
                    records.append({
                        "subject": subject['subject_name'],
                        "chapter": chapter['chapter_name'],
                        "subtopic": subtopic['subtopic_name'],
                        "question_type": q_type,
                        "question_text": question_text,
                        "options": options,
                        "correct_answer": correct_answer,
                        "difficulty": q.get("difficulty")
                    })

df = pd.DataFrame(records)
df.dropna(subset=['question_text', 'correct_answer', 'difficulty'], inplace=True)

In [14]:
# === Encode features and train Decision Tree ===
le_topic = LabelEncoder()
le_diff = LabelEncoder()
le_next = LabelEncoder()

df['topic_enc'] = le_topic.fit_transform(df['chapter'])
df['difficulty_enc'] = le_diff.fit_transform(df['difficulty'])
df['next_difficulty'] = df['difficulty'].shift(-1).fillna('easy')
df['next_diff_enc'] = le_next.fit_transform(df['next_difficulty'])

X = df[['topic_enc', 'difficulty_enc']]
y = df['next_diff_enc']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
clf = DecisionTreeClassifier(max_depth=5)
clf.fit(X_train, y_train)

# === TF-IDF + Cosine Similarity Setup ===
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['question_text'].astype(str))

In [ ]:
# === Function to find next similar question ===
def get_similar_question(current_question_text, predicted_difficulty, chapter):
    user_vec = vectorizer.transform([current_question_text])
    sims = cosine_similarity(user_vec, tfidf_matrix).flatten()

    candidates = df[(df['difficulty'] == predicted_difficulty) & (df['chapter'] == chapter)].copy()
    if candidates.empty:
        return None
    
    best_score = -1
    best_idx = -1

    for idx in candidates.index:
        sim = sims[idx]
        if sim > best_score:
            best_score = sim
            best_idx = idx

    return df.loc[best_idx] if best_idx != -1 else None

In [ ]:
# === Quiz UI with Loop ===
name = input("\U0001F44B Enter your name: ")
print("\n\U0001F4DA Available Chapters:")
chapters = df['chapter'].unique()
for i, chapter in enumerate(chapters, 1):
    print(f"{i}. {chapter}")

selected_idx = int(input("Select a chapter number: ")) - 1
selected_chapter = chapters[selected_idx]

score = 0
question_count = 0
current_difficulty = 'easy'
correct_streak = 0
streak_threshold = 3  # number of correct answers needed to level up

questions = df[df['chapter'] == selected_chapter].to_dict(orient='records')
total_questions = 20

while question_count < total_questions:
    eligible = [q for q in questions if q['difficulty'] == current_difficulty]
    if not eligible:
        print(f"No questions at {current_difficulty.upper()} level. Switching to 'medium'.")
        current_difficulty = 'medium'
        continue

    question = random.choice(eligible)
    print(f"\nQ{question_count+1} [{current_difficulty.upper()}]: {question['question_text']}")
    for j, opt in enumerate(question['options']):
        print(f"{j+1}. {opt}")
    user_ans = input("Your answer (1/2/3...): ").strip()

    try:
        chosen = question['options'][int(user_ans)-1].lower()
        correct = question['correct_answer'].lower()
        if correct in chosen:
            print("✅ Correct!")
            score += 1
            correct_streak += 1

            if correct_streak >= streak_threshold:
                if current_difficulty == 'easy':
                    current_difficulty = 'medium'
                elif current_difficulty == 'medium':
                    current_difficulty = 'hard'
                correct_streak = 0
        else:
            print(f"❌ Wrong! Correct answer is: {question['correct_answer']}")
            correct_streak = 0
            if current_difficulty == 'hard':
                current_difficulty = 'medium'
            elif current_difficulty == 'medium':
                current_difficulty = 'easy'

    except (ValueError, IndexError):
        print(f"❌ Invalid input! Correct answer is: {question['correct_answer']}")

    question_count += 1

print(f"\n🎉 Quiz finished, {name}! Your score: {score}/{total_questions}")


👋 Enter your name: asjai

📚 Available Chapters:
1. Physics and Measurements
2. Electronic Devices
Select a chapter number: 1

Q1 [EASY]: A measuring instrument with a smaller least count is generally more precise.
1. True
2. False
Your answer (1/2/3...): 1
✅ Correct!

Q2 [EASY]: What are the dimensions of kinetic energy?
1. MLT^-1
2. ML^2T^-2
3. MLT^-2
4. M^2LT^-2
Your answer (1/2/3...): 2
✅ Correct!

Q3 [EASY]: If the velocity (v) of a particle is dependent on time (t) as v = At + Bt^2, then the dimensions of B are ___.
1. [LT^-1]
2. [LT^-2]
3. [LT^-3]
4. [LT^-4]
Your answer (1/2/3...): 3
✅ Correct!

Q4 [MEDIUM]: The SI unit of mass is the gram.
1. True
2. False
Your answer (1/2/3...): 2
✅ Correct!

Q5 [MEDIUM]: A vernier caliper has 20 divisions on the vernier scale that coincide with 19 divisions on the main scale. If each main scale division is 1 mm, what is the least count of the caliper?
1. 0.01 mm
2. 0.05 mm
3. 0.1 mm
4. 1 mm
Your answer (1/2/3...): 2
✅ Correct!

Q6 [MEDIUM]: Wh